In [1]:
import cv2
import os
import tensorflow
from tensorflow.keras.preprocessing import image

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16

In [3]:
# load and evaluate a saved model
from numpy import loadtxt
from tensorflow.keras.models import load_model
 
# load model
model = load_model('model.h5')
# summarize model.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [4]:
def detect_face_mask(img):
  img = img.reshape(1,224,224,3)
  y_pred = (model.predict(img) > 0.5).astype("int32")
  return y_pred[0][0]

In [5]:
sample_withoutm = cv2.imread('sample\withoutmask.jpg')
sample_withoutm = cv2.resize(sample_withoutm, (224,224))

In [6]:
sample_withm = cv2.imread('sample\withmask.jpg')
sample_withm = cv2.resize(sample_withm, (224,224))

In [7]:
detect_face_mask(sample_withoutm) 

1

In [8]:
detect_face_mask(sample_withm) 

0

In [9]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
def detect(img):
    coods = faceCascade.detectMultiScale(img)
    return coods

In [10]:
def draw_label(img, text, pos, bg_color):
  text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)
  end_x = pos[0] + text_size[0][0] + 2
  end_y = pos[1] + text_size[0][1] - 2
  cv2. rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED)
  cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)

In [11]:
while True:
    ret, frame = cap.read()
    coods = detect(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    #prediction
    img = cv2.resize(frame, (224, 224))
    y_pred = detect_face_mask(img) 
    if y_pred == 0:
      draw_label(frame, "mask", (20,20), (0,255,0))
    else:
      draw_label(frame, "no mask", (20,20), (0,0,255))
    for(x, y, w, h) in coods:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    cv2.imshow('live', frame)
    if cv2.waitKey(1) &  0xFF == ord('x'):
        break
cv2.destroyAllWindows()